In [1]:
import numpy as np
import torch
from torch.utils import data

def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, torch.reshape(y, (-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[ 4.4630e-01, -1.4504e-01],
         [ 1.1572e+00, -1.6324e-01],
         [-1.7575e+00,  2.0445e-01],
         [-6.6655e-01, -2.2168e-01],
         [-3.1761e-01, -1.1592e+00],
         [-2.4185e-01, -3.4603e-01],
         [-1.0008e+00,  1.4873e-01],
         [-3.3180e-01, -1.8039e+00],
         [-2.1308e-01,  4.5083e-01],
         [ 4.1490e-04,  1.6481e+00]]),
 tensor([[ 5.5663e+00],
         [ 7.0634e+00],
         [-1.4588e-03],
         [ 3.6016e+00],
         [ 7.4976e+00],
         [ 4.8855e+00],
         [ 1.6980e+00],
         [ 9.6664e+00],
         [ 2.2288e+00],
         [-1.4005e+00]])]

In [4]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [6]:
loss = nn.MSELoss()

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for x, y in data_iter:
        l = loss(net(x), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000211
epoch 2, loss 0.000105
epoch 3, loss 0.000105


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([9.2983e-06, 5.8436e-04])
b的估计误差： tensor([0.0003])
